<a href="https://colab.research.google.com/github/dariashcherbakovaaa/MA_DSE_Unimi/blob/Algo/Spark_mr_baics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

All data for the project from Kaggle should be used via API,
no action from the teacher

In [6]:
import findspark
findspark.init()# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #Property used to format output tables batter
spark

In [7]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

### RDD resilient distributed dataset

- resilient (done by replicating chunks)
- distributed (there're several maschine in the file system)

When we permorm ***transformation*** on RDD, we transform RDD to another RDD, it returns RDD, while in ***action*** the result is something different stored in a memory, thus we should be awre of the output's size


In [8]:
sc = spark.sparkContext

In [9]:
s = '''Every person had a star, every star had a friend, and for every person
carrying a star there was someone else who reflected it, and everyone
carried this reflection like a secret confidante in the heart'''
simple_rdd = sc.parallelize(s.split('\n'))

In [10]:
simple_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

Here "*ParallelCollectionRDD[0]*" is an example of the ***action***

In [11]:
simple_rdd.collect()

['Every person had a star, every star had a friend, and for every person',
 'carrying a star there was someone else who reflected it, and everyone',
 'carried this reflection like a secret confidante in the heart']

The code below it the result of transformation

In [12]:
simple_rdd.map(lambda line: line.split(' '))

# def fun(x):
  # return expression

PythonRDD[1] at RDD at PythonRDD.scala:53

The codebelow is an example of ***transformation***

In [13]:
(simple_rdd.map(lambda line: line.split(' '))
           .collect())

[['Every',
  'person',
  'had',
  'a',
  'star,',
  'every',
  'star',
  'had',
  'a',
  'friend,',
  'and',
  'for',
  'every',
  'person'],
 ['carrying',
  'a',
  'star',
  'there',
  'was',
  'someone',
  'else',
  'who',
  'reflected',
  'it,',
  'and',
  'everyone'],
 ['carried',
  'this',
  'reflection',
  'like',
  'a',
  'secret',
  'confidante',
  'in',
  'the',
  'heart']]

In [17]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .take(17))

['Every',
 'person',
 'had',
 'a',
 'star,',
 'every',
 'star',
 'had',
 'a',
 'friend,',
 'and',
 'for',
 'every',
 'person',
 'carrying',
 'a',
 'star']

In [16]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(",", '').lower())
           .take(10))

['every', 'person', 'had', 'a', 'star', 'every', 'star', 'had', 'a', 'friend']

In [18]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(",", '').lower())
           .map(lambda word: (word, 1))
           .take(5))

[('every', 1), ('person', 1), ('had', 1), ('a', 1), ('star', 1)]

HADOOP = Map + Reduce (work with key-value pairs)

SPARK's reduce is different


In [19]:
(simple_rdd.flatMap(lambda line: line.split(' '))
           .map(lambda word: word.replace(",", '').lower())
           .map(lambda word: (word, 1))
           .reduceByKey(lambda a,b: a+b) # it's a trunsformation since it returns the set of tuples, RDD
           .collect())

[('person', 2),
 ('there', 1),
 ('was', 1),
 ('carried', 1),
 ('this', 1),
 ('like', 1),
 ('secret', 1),
 ('confidante', 1),
 ('in', 1),
 ('heart', 1),
 ('every', 3),
 ('had', 2),
 ('a', 4),
 ('star', 3),
 ('friend', 1),
 ('and', 2),
 ('for', 1),
 ('carrying', 1),
 ('someone', 1),
 ('else', 1),
 ('who', 1),
 ('reflected', 1),
 ('it', 1),
 ('everyone', 1),
 ('reflection', 1),
 ('the', 1)]

The communication cost (sorting) is remarkable

In [25]:
def count_freq(rdd):
  return (rdd.flatMap(lambda line: line.split(' '))
            .map(lambda word: word.replace(',', '').lower())
            .map(lambda word: (word, 1))
            .reduceByKey(lambda a,b: a+b)
            .sortBy(lambda a:a[1], False)
            .collect())

In [21]:
!wget http://www.scifiscripts.com/scripts/swd1_5-74.txt

--2024-01-31 14:37:51--  http://www.scifiscripts.com/scripts/swd1_5-74.txt
Resolving www.scifiscripts.com (www.scifiscripts.com)... 207.32.177.145
Connecting to www.scifiscripts.com (www.scifiscripts.com)|207.32.177.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203125 (198K) [text/plain]
Saving to: ‘swd1_5-74.txt’

swd1_5-74.txt       100%[===================>] 198.36K  1.04MB/s    in 0.2s    

2024-01-31 14:37:51 (1.04 MB/s) - ‘swd1_5-74.txt’ saved [203125/203125]



In [22]:
sw = sc.textFile('swd1_5-74.txt')

In [23]:
sw.take(10)

['The Star Wars',
 'by',
 'George Lucas',
 '',
 '',
 '',
 'Rough Draft [First of four major screenplay drafts]',
 'Lucasfilm Ltd.',
 '5/74',
 '']

In [26]:
count_freq(sw)[: 100]

[('', 3543),
 ('the', 2731),
 ('and', 929),
 ('a', 897),
 ('of', 744),
 ('to', 702),
 ('general', 459),
 ('is', 393),
 ('in', 359),
 ('-', 330),
 ('starkiller', 302),
 ('his', 289),
 ('on', 274),
 ('he', 243),
 ('are', 209),
 ('you', 197),
 ('with', 196),
 ('they', 177),
 ('as', 162),
 ('two', 161),
 ('into', 153),
 ('for', 146),
 ('it', 145),
 ('be', 145),
 ('han', 143),
 ('at', 137),
 ('i', 134),
 ('from', 134),
 ('out', 129),
 ('by', 125),
 ('an', 122),
 ('whitsun', 122),
 ('have', 119),
 ('one', 114),
 ('aquilae', 114),
 ('your', 106),
 ('but', 101),
 ('all', 95),
 ('we', 94),
 ('threepio', 92),
 ('through', 84),
 ('back', 83),
 ('princess', 82),
 ('this', 80),
 ('artwo', 80),
 ('their', 79),
 ('will', 79),
 ('small', 77),
 ('him', 75),
 ('up', 74),
 ('get', 73),
 ('fortress', 72),
 ('them', 71),
 ('there', 69),
 ('imperial', 69),
 ('down', 67),
 ('space', 66),
 ('over', 66),
 ('large', 65),
 ("it's", 64),
 ('then', 63),
 ("i'm", 63),
 ('not', 59),
 ('young', 58),
 ('before', 57),


In [27]:
import random

NUM_SAMPLES = 10**7

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = (sc.parallelize(range(0, NUM_SAMPLES))
           .filter(inside).count())

print('Pi is roughly {}'.format(4.0 * count / NUM_SAMPLES))

Pi is roughly 3.1414408
